In [1]:
import numpy as np
data = np.load("mcs_hw2_p3_data.npy")
x = data[:, :2]
y = data[:, 2]

In [20]:
import scipy.stats

def get_gradient_mu_t(beta, mu, sigma2, v):
    return (beta - mu) / sigma2

def get_gradient_logsigma2_t(beta, mu, sigma2, v):
    norm = beta - mu
    return (- 1 / sigma2 + norm * norm / (2 * sigma2 * sigma2)) * sigma2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def get_log_p(x, y, beta):
    eps = 0.0
    res = 0.0
    res += np.sum(y * np.log(eps + sigmoid(np.dot(x, beta))) + (1.0 - y) * np.log(eps + 1 - sigmoid(np.dot(x, beta))))
    res += np.sum(scipy.stats.norm.logpdf(beta, np.zeros(2), np.ones(2)))
    return res

def get_log_q(mu, sigma2, beta, v):
    res = np.sum(scipy.stats.norm.logpdf(beta, mu, np.sqrt(sigma2)))
    return res

import concurrent.futures



def elbo(x, y, mu, sigma2, dof):
    res = 0.0
    sample_size = 1024
    sample_beta = np.zeros(shape=(2, sample_size))
    sample_beta = np.random.normal(mu, np.sqrt(sigma2), size=[sample_size, mu.shape[0]])
    with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
        future_list = [executor.submit(get_log_p, x, y, beta) for beta in sample_beta]
        for future in concurrent.futures.as_completed(future_list):
            res += future.result()

    with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
        future_list = [executor.submit(get_log_q, mu, sigma2, beta, dof) for beta in sample_beta]
        for future in concurrent.futures.as_completed(future_list):
            res += future.result()
            
    return res / sample_size

In [33]:
def bbvi_cv(x, y, mu, sigma2, lr, n_iter, m, v, dof):
    sample_size = 64
    sample_beta = np.zeros(shape=(2, sample_size))
    sample_beta = np.random.normal(mu, np.sqrt(sigma2), size=[sample_size, mu.shape[0]])
    # update mu
    loss_mu = np.zeros(shape=[sample_size, mu.shape[0]])
    loss_logsigma2 = np.zeros(shape=[sample_size, sigma2.shape[0]])
    cv_mu = np.zeros(shape=[sample_size, mu.shape[0]])
    cv_sigma2 = np.zeros(shape=[sample_size, sigma2.shape[0]])
    for i in range(sample_size):
        loss_mu[i] = cv_mu[i] = get_gradient_mu_t(sample_beta[i], mu, sigma2, dof)
        loss_logsigma2[i] = cv_sigma2[i] = get_gradient_logsigma2_t(sample_beta[i], mu, sigma2, dof)
        log_p = get_log_p(x, y, sample_beta[i])
        log_q = get_log_q(mu, sigma2, sample_beta[i], dof)
        loss_mu[i] *= (log_p - log_q)
        loss_logsigma2[i] *= (log_p - log_q)
        
    cov_mu0 = np.cov(np.stack((cv_mu.T[0], loss_mu.T[0]), axis=0))
    a_mu0 = cov_mu0[0][1] / cov_mu0[0][0]
    cov_mu1 = np.cov(np.stack((cv_mu.T[1], loss_mu.T[1]), axis=0))
    a_mu1 = cov_mu1[0][1] / cov_mu1[0][0]
    cov_logsigma2 = np.cov(np.stack((cv_sigma2.T[0], loss_logsigma2.T[0]), axis=0))
    a_logsigma2 = cov_logsigma2[0][1] / cov_logsigma2[0][0]
    
    update_mu = np.mean(loss_mu, axis=0)
    update_logsigma2 = np.mean(loss_logsigma2, axis=0)
    update_h_mu = np.mean(cv_mu, axis=0) * [a_mu0, a_mu1]
    update_h_logsigma2 = np.mean(cv_sigma2, axis=0) * a_logsigma2
    
    var_mu = np.var(loss_mu - cv_mu * [a_mu0, a_mu1], axis=0)
    var_sigma = np.var(loss_logsigma2 - cv_sigma2 * a_logsigma2, axis=0)
    
    grad = np.concatenate([update_mu - update_h_mu, update_logsigma2 - update_h_logsigma2])

    m = 0.9 * m + 0.1 * grad
    v = 0.999 * v + 0.001 * np.power(grad, 2)
    
    m_hat = m / (1 - np.power(0.9, n_iter))
    v_hat = v / (1 - np.power(0.999, n_iter))
    
    update = m_hat / (np.sqrt(v_hat))
    
    mu += lr * update[:2]
    sigma2 = np.exp(np.log(sigma2) + lr * update[2:])
    return mu, sigma2, m, v, var_mu, var_sigma

In [37]:
def train_bbvi_cv(x, y, n_iter, dof):
    mu_list = []
    sigma2_list = []
    var_list = []
    mu = np.random.normal(size=2)
    sigma2 = np.power(np.random.normal(size=2), 2)
    lr = 1
    m = np.zeros(shape=4)
    v = np.zeros(shape=4)
    for i in range(n_iter):
        mu, sigma2, m, v, var_mu, var_sigma = bbvi_cv(x, y, mu, sigma2, lr, i + 1, m, v, dof)
        mu_list.append(mu.copy())
        sigma2_list.append(sigma2.copy())
        var_list.append([var_mu.copy(), var_sigma.copy()])
    return mu_list, sigma2_list, var_list

In [38]:
'''
res_list = []
for dof in range(1, 100):
    mu_bbvi_cv, sigma2_bbvi_cv, var_bbvi_cv = train_bbvi_cv(x, y, 400, dof)
    elbo_list = [elbo(x, y, mu_bbvi_cv[i], sigma2_bbvi_cv[i], dof) for i in range(390, 400)]
    res_list.append(np.mean(np.array(elbo_list)))
    print(dof , ": ", res_list[dof - 1])
'''

'\nres_list = []\nfor dof in range(1, 100):\n    mu_bbvi_cv, sigma2_bbvi_cv, var_bbvi_cv = train_bbvi_cv(x, y, 400, dof)\n    elbo_list = [elbo(x, y, mu_bbvi_cv[i], sigma2_bbvi_cv[i], dof) for i in range(390, 400)]\n    res_list.append(np.mean(np.array(elbo_list)))\n    print(dof , ": ", res_list[dof - 1])\n'

In [39]:
mu_bbvi_cv, sigma2_bbvi_cv, var_bbvi_cv = train_bbvi_cv(x, y, 1000, 1)

In [35]:
elbo(x, y, mu_bbvi_cv[-1], sigma2_bbvi_cv[-1], 1)

-4448.285145721739

In [36]:
sigma2_bbvi_cv[-10:]

[array([0.00142832, 0.00077899]),
 array([0.00141647, 0.00077461]),
 array([0.00140369, 0.0007682 ]),
 array([0.00140048, 0.00076696]),
 array([0.00139428, 0.00076355]),
 array([0.00138799, 0.00075944]),
 array([0.00138224, 0.00075325]),
 array([0.00137951, 0.00074935]),
 array([0.0013739 , 0.00074358]),
 array([0.00136833, 0.0007376 ])]